## 🦜🔗 LangChain RAG Agent (RAG nur bei Bedarf)


In [ ]:
from langchain_community.vectorstores.faiss import FAISS
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from helpers import llm, embeddings

## In dieser Folie schauen wir uns an, wie RAG als Tool funktioniert.

Es wird also nicht einfach auf gut Glück bei jeder Anfrage das RAG angeworfen und Dokumente mit in die Prompt gehängt. Das entscheidet unser Agent nun alleine.


#### Wir erstellen eine kleine in-memory FAISS Datenbank.


In [ ]:
loader = PyPDFLoader("LangChain.pdf")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
docs = text_splitter.split_documents(documents)
db = FAISS.from_documents(docs, embeddings())

#### Wir benutzen LangChain-Magie um aus der Vektor-Datenbank ein Tool zu machen, das der Agent benutzen kann

Die Beschreibung "This is the best place to look for any information about LangChain." ist essentiell für die Funktionalität der gesamten App. Damit weiß das LLM, dass es genau dieses Tool aufrufen muss, wenn es Informationen zu LangChain braucht.


In [ ]:
retriever = db.as_retriever()
tool = create_retriever_tool(
    retriever,
    "search_langchain_documentation",
    "This is the best place to look for any information about LangChain.",
)
tools = [tool]

#### Wir bauen den Agenten mit Hilfe von LangGraph prebuilt und geben ihm das Tool.

Das Prompt ziehen wir vom Hub. Es ist nicht sonderlich relevant.


In [ ]:
from langgraph.prebuilt import create_agent_executor
from langchain.agents import create_tool_calling_agent
from langchain import hub

prompt = hub.pull("reactagent/rag")
agent_runnable = create_tool_calling_agent(llm(), tools, prompt)
app = create_agent_executor(agent_runnable, tools)

#### Wir definieren eine kleine Funktion, die die Ausgabe formatiert.

Diese Arbeit muss man üblicherweise selbst tun, weil LangChain ja nicht weiß, welches Format man am Ende braucht.


In [ ]:
def formatter(iterator):
    for chunk in iterator:
        for k, v in chunk.items():
            yield f"{k}: {v}"


chain = app | formatter

In [ ]:
from langchain_core.messages import HumanMessage
inputs = {
    "input": [
        HumanMessage(content="How do I use LangChain with a Vectorstore?")
    ]
}

for chunk in chain.stream(inputs):
    print(chunk)

#### Echtes Streaming


In [ ]:
from helpers import graph_agent_llm_output_streamer_events
inputs = {
    "input": [
        HumanMessage(content="How do I use LangChain with a Vectorstore?")
    ]
}

await graph_agent_llm_output_streamer_events(app, inputs)